In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path= "drive/My Drive/SUTD/Year 3/CDS/Project/Team No Name/"
# root_path= "drive/My Drive/SUTD ML/Team No Name/"

In [ ]:
import os
import pandas as pd
import numpy as np
import keras
import pickle
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

csv_path = os.path.join(root_path,'UrbanSound8K/metadata/UrbanSound8K.csv')
spectrograms_path = os.path.join(root_path,'numpySpectrograms/')
test_size = 0.2
batch_size = 16

#Using original spectrograms

In [ ]:
from tqdm import tqdm

class SpecLoader():
  def __init__(self, x_set, spec_dir):
    self.x = x_set
    self.spec_dir = spec_dir

  def __len__(self):
    return len(self.x)

  def getitem(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return np.array(specs)

  def getitemlist(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return specs

class SpecLoaderWnn(keras.utils.Sequence):
  def __init__(self, x_set, batch_size, spec_dir):
    self.x = x_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs)

In [ ]:
data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]

Xtrainval, X_test, Ytrainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)

In [ ]:
#Xtrainval = data_df_known['slice_file_name'].tolist()
#Ytrainval = data_df_known['classID'].tolist()

sl = SpecLoader(Xtrainval, spectrograms_path)
trainX = sl.getitem()

nsamples, x, y, z = trainX.shape
trainX = trainX.reshape((nsamples, x*y*z))

In [ ]:

# define outlier detection model
modellinear = OneClassSVM(kernel="linear", verbose=True)

# fit 
modellinear.fit(trainX)

# save
pickle.dump(modellinear, open(root_path+"OneClassSVM Models/original_linear", 'wb'))

[LibSVM]

In [ ]:
modelpoly = OneClassSVM(kernel="poly", verbose=True)

# fit 
modelpoly.fit(trainX)

# save
pickle.dump(modelpoly, open(root_path+"OneClassSVM Models/original_poly", 'wb'))

[LibSVM]

In [ ]:
modelrbf = OneClassSVM(kernel="rbf", verbose=True)

# fit 
modelrbf.fit(trainX)

# save
pickle.dump(modelrbf, open(root_path+"OneClassSVM Models/original_rbf", 'wb'))

[LibSVM]

In [ ]:
modelsigmoid = OneClassSVM(kernel="sigmoid", verbose=True)

# fit 
modelsigmoid.fit(trainX)

# save
pickle.dump(modelsigmoid, open(root_path+"OneClassSVM Models/original_sigmoid", 'wb'))

[LibSVM]

###Test

In [ ]:
# load in the models
print("loading linear...")
modellinear = pickle.load(open(root_path+"OneClassSVM Models/original_linear", 'rb'))
print("loading poly...")
modelpoly = pickle.load(open(root_path+"OneClassSVM Models/original_poly", 'rb'))
print("loading rbf...")
modelrbf = pickle.load(open(root_path+"OneClassSVM Models/original_rbf", 'rb'))
print("loading sigmoid...")
modelsigmoid = pickle.load(open(root_path+"OneClassSVM Models/original_sigmoid", 'rb'))

loading linear...
loading poly...
loading rbf...
loading sigmoid...


In [ ]:
#Xtestvalunknown = data_df_unknown['slice_file_name'].tolist()
X_trash, Xtestvalunknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)

#Xtestvalunknown = Xtestvalunknown[:25]
#X_test = X_test[:25]

testX = Xtestvalunknown + X_test

actualtestY = [-1 for i in Xtestvalunknown] + y_test
print("length of actual test y", len(actualtestY))

testY = [-1 for i in Xtestvalunknown]+[1 for i in X_test]
testY = np.array(testY)
print(testY.shape)
print("length of test y (1 and -1)", len(testY))

sltest = SpecLoader(testX, spectrograms_path)
testX = sltest.getitem()

nsamples, x, y, z = testX.shape
testX = testX.reshape((nsamples, x*y*z))

  0%|          | 0/1747 [00:00<?, ?it/s]

length of actual test y 1747
(1747,)
length of test y (1 and -1) 1747


100%|██████████| 1747/1747 [09:53<00:00,  2.95it/s]


In [ ]:
# detect outliers in the test set
print("predicting linear...")
yhatLinear = modellinear.predict(testX)
print("predicting poly...")
yhatPoly = modelpoly.predict(testX)
print("predicting rbf...")
yhatRBF = modelrbf.predict(testX)
print("predicting sigmoid...")
yhatSigmoid = modelsigmoid.predict(testX)

predicting linear...
predicting poly...
predicting rbf...
predicting sigmoid...


In [ ]:
# evaluate
# input: x,y - numpy of shape (n,) 
# output: percentage of hits
def evaluate(x,y):
  hit = 0
  for i in range(len(y)):
    if int(x[i])==int(y[i]):
      hit+=1
  return hit/len(y)

def evaluate2(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))
    for average in ["macro", "weighted", "micro"]:
        f1 = f1_score(expected, predicted, average=average)
        print("{} f1 score: {}".format(average, f1))

In [ ]:
from sklearn.metrics import f1_score

kn = ["Linear","Poly","RBF","Sigmoid"]
yhats = [yhatLinear,yhatPoly,yhatRBF,yhatSigmoid]
print("Accuracy - hit rate")
print("--------------------------------")

for i in range(len(kn)):
  print("{}: {}".format(kn[i],evaluate(yhats[i],testY)))

Accuracy - hit rate
--------------------------------
Linear: 0.527761877504293
Poly: 0.5037206639954207
RBF: 0.46307956496851743
Sigmoid: 0.5317687464224384


In [ ]:
from tensorflow import keras 
model = keras.models.load_model(root_path+"basemodel-known")


In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatLinear)) if yhatLinear[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatLinear[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatLinear
  yhatLinear[index_yhat] = predClass

print("prediction of positive classes through base model", yhatLinear)

print("\nLinear - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatLinear, average='macro'))
print("Micro:", f1_score(actualtestY, yhatLinear, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatLinear, average='weighted'))
print("None:", f1_score(actualtestY, yhatLinear, average=None))

print("\nEvaluate\n")
evaluate2(yhatLinear, actualtestY)

length of index_normalList 859
length of normalXfilenames 859
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
predict using one class svm: [ 1 -1 -1  1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1  1  1  1  1  1 -1 -1  1  1
  1 -1  1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1  1  1  1  1 -1  1  1  1 -1
 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1  1  1 -1 -1
  1  1  1 -1 -1  1  1 -1  1 -1  1 -1  1  1 -1  1  1 -1  1 -1 -1 -1  1  1
 -1 -1 -1 -1]
prediction of positive classes through base model [ 4 -1 -1 ... -1  2 -1]

Linear - F1 Scores
--------------------------------
Macro: 0.4823245617287202
Micro: 0.5123068116771609
Weighted: 0.5047722601774953
None: [0.52830189 0.4040404  0.47945205 0.61691542 0.5891182 

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatPoly)) if yhatPoly[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatPoly[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatPoly
  yhatPoly[index_yhat] = predClass

print("prediction of positive classes through base model", yhatPoly)

print("\nPoly - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatPoly, average='macro'))
print("Micro:", f1_score(actualtestY, yhatPoly, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatPoly, average='weighted'))
print("None:", f1_score(actualtestY, yhatPoly, average=None))

print("\nEvaluate\n")
evaluate2(yhatPoly, actualtestY)

length of index_normalList 897
length of normalXfilenames 897
predict using one class svm: [ 1  1 -1  1  1  1 -1  1 -1  1 -1 -1  1 -1 -1  1  1  1  1  1 -1 -1 -1  1
  1 -1  1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1  1  1  1  1  1  1 -1
 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1  1  1 -1  1  1 -1 -1
  1  1  1 -1 -1  1  1 -1  1 -1  1 -1  1  1 -1  1  1  1  1 -1  1 -1  1  1
 -1 -1 -1 -1]
prediction of positive classes through base model [ 4  4 -1 ... -1 -1 -1]

Poly - F1 Scores
--------------------------------
Macro: 0.4600097340004173
Micro: 0.48769318832283914
Weighted: 0.48250618399920786
None: [0.49327878 0.41089109 0.38888889 0.58974359 0.58866545 0.2885906 ]

Evaluate

Overall accuracy: 0.48769318832283914
Accuracy for class -1: 0.4901277584204414
Accuracy for class 2: 0.5665024630541872
Accuracy for class 4: 0.25
Accuracy for class 0: 0.39712918660287083
Accuracy for class 3: 0.706140350877193
Accuracy for class 1: 0.3783783783783784
macro f1 score: 0.4600097340004173
weig

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatRBF)) if yhatRBF[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatRBF[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatRBF
  yhatRBF[index_yhat] = predClass

print("prediction of positive classes through base model", yhatRBF)

print("\nRBF - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatRBF, average='macro'))
print("Micro:", f1_score(actualtestY, yhatRBF, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatRBF, average='weighted'))
print("None:", f1_score(actualtestY, yhatRBF, average=None))

print("\nEvaluate\n")
evaluate2(yhatRBF, actualtestY)

length of index_normalList 944
length of normalXfilenames 944
predict using one class svm: [-1  1  1  1 -1  1  1 -1  1 -1  1  1 -1 -1  1 -1 -1 -1 -1  1  1 -1  1 -1
 -1  1  1  1  1  1  1  1 -1  1  1  1  1 -1 -1  1 -1  1 -1 -1 -1  1  1  1
  1  1  1  1  1 -1  1 -1  1  1 -1  1 -1  1  1  1  1  1  1 -1  1 -1 -1  1
  1 -1 -1 -1 -1 -1 -1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1
  1  1  1  1]
prediction of positive classes through base model [-1  4  0 ...  3  2 -1]

RBF - F1 Scores
--------------------------------
Macro: 0.44533123971837707
Micro: 0.44934172867773325
Weighted: 0.4430046915041052
None: [0.43629808 0.58264463 0.44776119 0.66968326 0.27322404 0.26237624]

Evaluate

Overall accuracy: 0.44934172867773325
Accuracy for class -1: 0.42160278745644597
Accuracy for class 2: 0.729064039408867
Accuracy for class 4: 0.3081395348837209
Accuracy for class 0: 0.6746411483253588
Accuracy for class 3: 0.21929824561403508
Accuracy for class 1: 0.40540540540540543
macro f1 score: 0.445331

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatSigmoid)) if yhatSigmoid[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatSigmoid[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatSigmoid
  yhatSigmoid[index_yhat] = predClass

print("prediction of positive classes through base model", yhatSigmoid)

print("\nSigmoid - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatSigmoid, average='macro'))
print("Micro:", f1_score(actualtestY, yhatSigmoid, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatSigmoid, average='weighted'))
print("None:", f1_score(actualtestY, yhatSigmoid, average=None))

print("\nEvaluate\n")
evaluate2(yhatSigmoid, actualtestY)

length of index_normalList 866
length of normalXfilenames 866
predict using one class svm: [ 1 -1  1  1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1  1  1  1  1 -1 -1  1
  1 -1  1  1 -1 -1 -1  1  1 -1 -1 -1 -1 -1  1  1  1  1  1 -1 -1  1  1 -1
 -1 -1  1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1  1 -1 -1
  1 -1 -1 -1 -1  1 -1  1  1 -1  1  1  1 -1 -1 -1  1 -1  1 -1 -1 -1 -1  1
  1 -1 -1 -1]
prediction of positive classes through base model [ 4 -1  0 ...  3  2 -1]

Sigmoid - F1 Scores
--------------------------------
Macro: 0.4875586054491046
Micro: 0.5151688609044076
Weighted: 0.5087364078889922
None: [0.5304248  0.43520782 0.47222222 0.66059226 0.53278689 0.29411765]

Evaluate

Overall accuracy: 0.5151688609044076
Accuracy for class -1: 0.5365853658536586
Accuracy for class 2: 0.7142857142857143
Accuracy for class 4: 0.23255813953488372
Accuracy for class 0: 0.4258373205741627
Accuracy for class 3: 0.5701754385964912
Accuracy for class 1: 0.4594594594594595
macro f1 score: 0.487558

#Using features extracted from base model

In [ ]:
import numpy as np
import pandas as pd

import json
import keras
import tensorflow
from keras import layers
from keras.models import Sequential
from keras.applications import DenseNet201
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

from keras.models import load_model

In [ ]:
model_save_path = os.path.join(root_path,"basemodel-known")
# load model
model = load_model(model_save_path)
# summarize model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, None, None, 1920)  18321984  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 9605      
Total params: 18,331,589
Trainable params: 18,102,533
Non-trainable params: 229,056
_________________________________________________________________


In [ ]:
layer_names = [layer.name for layer in model.layers]
print(layer_names) #['densenet201', 'global_average_pooling2d_1', 'dense_1']

['densenet201', 'global_average_pooling2d_1', 'dense_1']


##Try using one training instance

In [ ]:
#try one data
onedatanp = np.load(spectrograms_path + "344-3-0-0.wav.npy")
print(onedatanp.shape)

#append numpy into [] -> 4 dimensions (nsamples,x,y,z)
finalonedatanp = []
finalonedatanp.append(onedatanp.transpose())
finalonedatanp = np.array(finalonedatanp)

print(finalonedatanp.shape)

(3, 128, 250)
(1, 250, 128, 3)


In [ ]:
from keras import Model

model_output = model.get_layer("global_average_pooling2d_1").output
m = Model(inputs=model.input, outputs=model_output)
finaloutput = m.predict(finalonedatanp)
print(finaloutput.shape)

(1, 1920)


In [ ]:
print(np.array(finaltrainingset).shape)

(128, 1920)


In [ ]:
np1 = np.array([[[1,2],[2,3,4,5]],[[3,4],[3,4,5,6]]])
print(np1.shape)
np2 = np.array([[[1,2],[2,3,4,5]],[[3,4],[3,4,5,6]]])
np1=np.concatenate((np1,np2),axis=0)
print(np1.shape)

(2, 2)
(4, 2)


In [ ]:
print(type(np.array([2,3])))

<class 'numpy.ndarray'>


##All training instances

In [ ]:
# original, shape into OneClassSVM: (3543, 96000)
# after NN, shape into OneClassSVM: (3543, 1920)

batch_size = 16

In [ ]:
class SpecLoaderWnn(keras.utils.Sequence):
  def __init__(self, x_set, batch_size, spec_dir):
    self.x = x_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def getitem(self,idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs)

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs)

In [ ]:
#prepare data
from sklearn.model_selection import train_test_split

data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]

Xtrainval, X_test, Ytrainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)

In [ ]:
#Xtrainval = data_df_known['slice_file_name'].tolist()
#Ytrainval = data_df_known['classID'].tolist()

train_loader = SpecLoaderWnn(Xtrainval, Ytrainval, batch_size, spectrograms_path)


In [ ]:
from keras import Model
from tqdm import tqdm
finaltrainingset = None
for ind in tqdm(range(len(train_loader))):
  model_output = model.get_layer("global_average_pooling2d_1").output
  m = Model(inputs=model.input, outputs=model_output)
  finaloutput = m.predict(train_loader.getitem(ind))
  if ind==0:
    print(finaloutput.shape)
  if type(finaltrainingset) == type(None):
    finaltrainingset = finaloutput
  else:
    finaltrainingset=np.concatenate((finaltrainingset,finaloutput),axis=0)





  0%|          | 0/222 [00:00<?, ?it/s]





  0%|          | 1/222 [00:05<18:54,  5.13s/it]

(16, 1920)






  1%|          | 2/222 [00:10<18:49,  5.13s/it]





  1%|▏         | 3/222 [00:15<18:33,  5.09s/it]

...





 98%|█████████▊| 218/222 [18:26<00:20,  5.21s/it]





 99%|█████████▊| 219/222 [18:31<00:15,  5.13s/it]





 99%|█████████▉| 220/222 [18:36<00:10,  5.02s/it]





100%|█████████▉| 221/222 [18:41<00:04,  4.93s/it]





100%|██████████| 222/222 [18:44<00:00,  5.07s/it]


In [ ]:
print(finaltrainingset.shape) #(3543, 1920)

In [ ]:
from sklearn.svm import OneClassSVM
# define outlier detection model
modellinear = OneClassSVM(kernel="linear", verbose=True)

# fit 
modellinear.fit(finaltrainingset)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='linear',
            max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=True)

In [ ]:
modelpoly = OneClassSVM(kernel="poly", verbose=True)

# fit 
modelpoly.fit(finaltrainingset)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='poly',
            max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=True)

In [ ]:
modelrbf = OneClassSVM(kernel="rbf", verbose=True)

# fit 
modelrbf.fit(finaltrainingset)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
            max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=True)

In [ ]:
modelsigmoid = OneClassSVM(kernel="sigmoid", verbose=True)

# fit 
modelsigmoid.fit(finaltrainingset)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale',
            kernel='sigmoid', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
            verbose=True)

In [ ]:
import pickle

kn = ["linear","poly","rbf","sigmoid"]
models = [modellinear,modelpoly,modelrbf,modelsigmoid]

# save model
for i in range(len(kn)):
  save_filename = os.path.join(root_path,"OneClassSVM Models/basemodel_"+kn[i])
  # save the model to disk
  pickle.dump(models[i], open(save_filename, 'wb'))

###Test

In [ ]:
# load in the models
print("loading linear...")
modellinear = pickle.load(open(root_path+"OneClassSVM Models/basemodel_linear", 'rb'))
print("loading poly...")
modelpoly = pickle.load(open(root_path+"OneClassSVM Models/basemodel_poly", 'rb'))
print("loading rbf...")
modelrbf = pickle.load(open(root_path+"OneClassSVM Models/basemodel_rbf", 'rb'))
print("loading sigmoid...")
modelsigmoid = pickle.load(open(root_path+"OneClassSVM Models/basemodel_sigmoid", 'rb'))

loading linear...
loading poly...
loading rbf...
loading sigmoid...


In [ ]:
#Xtestvalunknown = data_df_unknown['slice_file_name'].tolist()
X_trash, Xtestvalunknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)

#Xtestvalunknown = Xtestvalunknown[:25]
#X_test = X_test[:25]

testX = Xtestvalunknown + X_test

actualtestY = [-1 for i in Xtestvalunknown] + y_test
print("length of actual test y", len(actualtestY))

actualtestY = [-1 for i in Xtestvalunknown] + y_test
print("length of actual test y", len(actualtestY))

testY = [-1 for i in Xtestvalunknown]+[1 for i in X_test]
testY = np.array(testY)
print(testY.shape)

length of actual test y 1747
length of actual test y 1747
(1747,)


In [ ]:
from keras import Model
from tqdm import tqdm

test_loader = SpecLoaderWnn(testX, batch_size, spectrograms_path)

finaltestset = None
for ind in tqdm(range(len(test_loader))):
  model_output = model.get_layer("global_average_pooling2d_1").output
  m = Model(inputs=model.input, outputs=model_output)
  finaloutput = m.predict(test_loader.getitem(ind))
  if ind==0:
    print(finaloutput.shape)
  if type(finaltestset) == type(None):
    finaltestset = finaloutput
  else:
    finaltestset=np.concatenate((finaltestset,finaloutput),axis=0)

  1%|          | 1/110 [00:05<09:21,  5.16s/it]

(16, 1920)


  3%|▎         | 3/110 [00:16<09:32,  5.35s/it]

  4%|▎         | 4/110 [00:21<09:16,  5.25s/it]

  5%|▍         | 5/110 [00:25<09:01,  5.15s/it]

...

 96%|█████████▋| 106/110 [09:34<00:22,  5.57s/it]

 97%|█████████▋| 107/110 [09:39<00:16,  5.44s/it]

 98%|█████████▊| 108/110 [09:45<00:11,  5.67s/it]

 99%|█████████▉| 109/110 [09:50<00:05,  5.54s/it]

100%|██████████| 110/110 [09:54<00:00,  5.40s/it]


In [ ]:
print(finaltestset.shape) #(1747, 1920)

(1747, 1920)


In [ ]:
# detect outliers in the test set
print("predicting linear...")
yhatLinear = modellinear.predict(finaltestset)
print("predicting poly...")
yhatPoly = modelpoly.predict(finaltestset)
print("predicting rbf...")
yhatRBF = modelrbf.predict(finaltestset)
print("predicting sigmoid...")
yhatSigmoid = modelsigmoid.predict(finaltestset)

predicting linear...
predicting poly...
predicting rbf...
predicting sigmoid...


In [ ]:
print(yhatLinear.shape)

(1747,)


In [ ]:
# evaluate
# input: x,y - numpy of shape (n,) 
# output: percentage of hits
def evaluate(x,y):
  hit = 0
  for i in range(len(y)):
    if int(x[i])==int(y[i]):
      hit+=1
  return hit/len(y)

def evaluate2(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))
    for average in ["macro", "weighted", "micro"]:
        f1 = f1_score(expected, predicted, average=average)
        print("{} f1 score: {}".format(average, f1))

In [ ]:
kn = ["Linear","Poly","RBF","Sigmoid"]
yhats = [yhatLinear,yhatPoly,yhatRBF,yhatSigmoid]
print("Accuracy - hit rate")
print("--------------------------------")

for i in range(len(kn)):
  print("{}: {}".format(kn[i],evaluate(yhats[i],testY)))


Accuracy - hit rate
--------------------------------
Linear: 0.683457355466514
Poly: 0.6886090440755581
RBF: 0.3331425300515169
Sigmoid: 0.6794504865483686


In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatLinear)) if yhatLinear[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatLinear[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatLinear
  yhatLinear[index_yhat] = predClass

print("prediction of positive classes through base model", yhatLinear)

print("\nLinear - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatLinear, average='macro'))
print("Micro:", f1_score(actualtestY, yhatLinear, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatLinear, average='weighted'))
print("None:", f1_score(actualtestY, yhatLinear, average=None))

print("\nEvaluate\n")
evaluate2(yhatLinear, actualtestY)

length of index_normalList 505
length of normalXfilenames 505
predict using one class svm: [-1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
prediction of positive classes through base model [-1  4 -1 ... -1 -1 -1]

Linear - F1 Scores
--------------------------------
Macro: 0.6233993776581975
Micro: 0.67830566685747
Weighted: 0.6598884054226275
None: [0.73704232 0.53246753 0.67241379 0.59333333 0.52601156 0.67912773]

Evaluate

Overall accuracy: 0.67830566685747
Accuracy for class -1: 0.9001161440185831
Accuracy for class 2: 0.43842364532019706
Accuracy for class 4: 0.6337209302325582
Accuracy for class 0: 0.3923444976076555
Accuracy for class 3: 0.3991228070175439
Accuracy for class 1: 0.527027027027027
macro f1 score: 0.623399377658

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatPoly)) if yhatPoly[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatPoly[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatPoly
  yhatPoly[index_yhat] = predClass

print("prediction of positive classes through base model", yhatPoly)

print("\nPoly - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatPoly, average='macro'))
print("Micro:", f1_score(actualtestY, yhatPoly, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatPoly, average='weighted'))
print("None:", f1_score(actualtestY, yhatPoly, average=None))

print("\nEvaluate\n")
evaluate2(yhatPoly, actualtestY)

length of index_normalList 482
length of normalXfilenames 482
predict using one class svm: [-1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
prediction of positive classes through base model [-1  4 -1 ... -1 -1 -1]

Poly - F1 Scores
--------------------------------
Macro: 0.6267071185604906
Micro: 0.6846021751574127
Weighted: 0.6653804941979486
None: [0.74412041 0.57413249 0.67241379 0.62987013 0.50154799 0.63815789]

Evaluate

Overall accuracy: 0.6846021751574127
Accuracy for class -1: 0.9186991869918699
Accuracy for class 2: 0.47783251231527096
Accuracy for class 4: 0.563953488372093
Accuracy for class 0: 0.4354066985645933
Accuracy for class 3: 0.35526315789473684
Accuracy for class 1: 0.527027027027027
macro f1 score: 0.6267071185

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatRBF)) if yhatRBF[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatRBF[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatRBF
  yhatRBF[index_yhat] = predClass

print("prediction of positive classes through base model", yhatRBF)

print("\nRBF - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatRBF, average='macro'))
print("Micro:", f1_score(actualtestY, yhatRBF, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatRBF, average='weighted'))
print("None:", f1_score(actualtestY, yhatRBF, average=None))

print("\nEvaluate\n")
evaluate2(yhatRBF, actualtestY)

length of index_normalList 1217
length of normalXfilenames 1217
predict using one class svm: [ 1 -1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1 -1  1  1 -1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1 -1  1  1 -1  1
  1  1  1  1  1  1  1 -1  1 -1  1  1 -1  1  1  1  1  1  1  1  1  1 -1 -1
  1  1  1 -1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1]
prediction of positive classes through base model [ 4 -1  0 ...  3  2 -1]

RBF - F1 Scores
--------------------------------
Macro: 0.3466274763647119
Micro: 0.3113909559244419
Weighted: 0.288500974605474
None: [0.16247304 0.41532258 0.20437956 0.50731707 0.41577061 0.37450199]

Evaluate

Overall accuracy: 0.3113909559244419
Accuracy for class -1: 0.13124274099883856
Accuracy for class 2: 0.5123152709359606
Accuracy for class 4: 0.5465116279069767
Accuracy for class 0: 0.49282296650717705
Accuracy for class 3: 0.5087719298245614
Accuracy for class 1: 0.1891891891891892
macro f1 score: 0.34662747

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatSigmoid)) if yhatSigmoid[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatSigmoid[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatSigmoid
  yhatSigmoid[index_yhat] = predClass

print("prediction of positive classes through base model", yhatSigmoid)

print("\nSigmoid - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatSigmoid, average='macro'))
print("Micro:", f1_score(actualtestY, yhatSigmoid, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatSigmoid, average='weighted'))
print("None:", f1_score(actualtestY, yhatSigmoid, average=None))

print("\nEvaluate\n")
evaluate2(yhatSigmoid, actualtestY)

length of index_normalList 504
length of normalXfilenames 504
predict using one class svm: [-1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
prediction of positive classes through base model [-1  4 -1 ... -1 -1 -1]

Sigmoid - F1 Scores
--------------------------------
Macro: 0.6178692444865497
Micro: 0.6742987979393246
Weighted: 0.6546729644028844
None: [0.7338403  0.50331126 0.67241379 0.56462585 0.53977273 0.69325153]

Evaluate

Overall accuracy: 0.6742987979393246
Accuracy for class -1: 0.8966318234610917
Accuracy for class 2: 0.4088669950738916
Accuracy for class 4: 0.6569767441860465
Accuracy for class 0: 0.36363636363636365
Accuracy for class 3: 0.4166666666666667
Accuracy for class 1: 0.527027027027027
macro f1 score: 0.6178692

#Using features extracted from VGGish

In [ ]:
from tqdm import tqdm

class SpecLoader():
  def __init__(self, x_set, spec_dir):
    self.x = x_set
    self.spec_dir = spec_dir

  def __len__(self):
    return len(self.x)

  def getitem(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return np.array(specs)

  def getitemlist(self):
    specs = []
    for fileName in tqdm(self.x):
        spec = np.load(self.spec_dir + fileName + ".npy")
        specs.append(spec.transpose())
    return specs

class SpecLoaderWnn(keras.utils.Sequence):
  def __init__(self, x_set, batch_size, spec_dir):
    self.x = x_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    # batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs)

In [ ]:
# original, shape into OneClassSVM: (3543, 96000)
# after VGGish, shape into OneClassSVM: (3543, 128)

batch_size = 16

In [ ]:
from sklearn.model_selection import train_test_split

vggfeatures_path = os.path.join(root_path,"embeddings/")

data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]

Xtrainval, X_test, Ytrainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)


In [ ]:
#Xtrainval = data_df_known['slice_file_name'].tolist()
#Ytrainval = data_df_known['classID'].tolist()

slvgg = SpecLoader(Xtrainval, vggfeatures_path)
trainX = slvgg.getitem()

print(trainX.shape)



100%|██████████| 3543/3543 [29:08<00:00,  2.03it/s]

(3543, 128, 1)


In [ ]:
nsamples, x, y = trainX.shape
trainX = trainX.reshape((nsamples, x*y))

In [ ]:
from sklearn.svm import OneClassSVM
# define outlier detection model
modellinear = OneClassSVM(kernel="linear", verbose=True)

# fit 
modellinear.fit(trainX)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='linear',
            max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=True)

In [ ]:
modelpoly = OneClassSVM(kernel="poly", verbose=True)

# fit 
modelpoly.fit(trainX)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='poly',
            max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=True)

In [ ]:
modelrbf = OneClassSVM(kernel="rbf", verbose=True)

# fit 
modelrbf.fit(trainX)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
            max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=True)

In [ ]:
modelsigmoid = OneClassSVM(kernel="sigmoid", verbose=True)

# fit 
modelsigmoid.fit(trainX)

[LibSVM]

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale',
            kernel='sigmoid', max_iter=-1, nu=0.5, shrinking=True, tol=0.001,
            verbose=True)

In [ ]:
import pickle

kn = ["linear","poly","rbf","sigmoid"]
models = [modellinear,modelpoly,modelrbf,modelsigmoid]

# save model
for i in range(len(kn)):
  save_filename = os.path.join(root_path,"OneClassSVM Models/vgg_"+kn[i])
  # save the model to disk
  pickle.dump(models[i], open(save_filename, 'wb'))

##Test

In [ ]:
# load in the models
print("loading linear...")
modellinear = pickle.load(open(root_path+"OneClassSVM Models/vgg_linear", 'rb'))
print("loading poly...")
modelpoly = pickle.load(open(root_path+"OneClassSVM Models/vgg_poly", 'rb'))
print("loading rbf...")
modelrbf = pickle.load(open(root_path+"OneClassSVM Models/vgg_rbf", 'rb'))
print("loading sigmoid...")
modelsigmoid = pickle.load(open(root_path+"OneClassSVM Models/vgg_sigmoid", 'rb'))

loading linear...
loading poly...
loading rbf...
loading sigmoid...


In [ ]:
#Xtestvalunknown = data_df_unknown['slice_file_name'].tolist()
X_trash, Xtestvalunknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)

#Xtestvalunknown = Xtestvalunknown[:25]
#X_test = X_test[:25]

testX = Xtestvalunknown + X_test

actualtestY = [-1 for i in Xtestvalunknown] + y_test
print("length of actual test y", len(actualtestY))

testY = [-1 for i in Xtestvalunknown]+[1 for i in X_test]
testY = np.array(testY)
print(testY.shape)

sltest = SpecLoader(testX, vggfeatures_path)
testX = sltest.getitem()

nsamples, x, y = testX.shape
testX = testX.reshape((nsamples, x*y))

  0%|          | 0/1747 [00:00<?, ?it/s]

length of actual test y 1747
(1747,)


100%|██████████| 1747/1747 [08:43<00:00,  3.34it/s]


In [ ]:
# detect outliers in the test set
yhatLinear = modellinear.predict(testX)
yhatPoly = modelpoly.predict(testX)
yhatRBF = modelrbf.predict(testX)
yhatSigmoid = modelsigmoid.predict(testX)

In [ ]:
# evaluate
# input: x,y - numpy of shape (n,) 
# output: percentage of hits
def evaluate(x,y):
  hit = 0
  for i in range(len(y)):
    if int(x[i])==int(y[i]):
      hit+=1
  return hit/len(y)

def evaluate2(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))
    for average in ["macro", "weighted", "micro"]:
        f1 = f1_score(expected, predicted, average=average)
        print("{} f1 score: {}".format(average, f1))

In [ ]:
kn = ["Linear","Poly","RBF","Sigmoid"]
yhats = [yhatLinear,yhatPoly,yhatRBF,yhatSigmoid]
print("Accuracy - hit rate")
print("--------------------------------")

for i in range(len(kn)):
  print("{}: {}".format(kn[i],evaluate(yhats[i],testY)))


Accuracy - hit rate
--------------------------------
Linear: 0.5031482541499713
Poly: 0.5054378935317687
RBF: 0.5151688609044076
Sigmoid: 0.4968517458500286


In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatLinear)) if yhatLinear[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatLinear[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatLinear
  yhatLinear[index_yhat] = predClass

print("prediction of positive classes through base model", yhatLinear)

print("\nLinear - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatLinear, average='macro'))
print("Micro:", f1_score(actualtestY, yhatLinear, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatLinear, average='weighted'))
print("None:", f1_score(actualtestY, yhatLinear, average=None))

print("\nEvaluate\n")
evaluate2(yhatLinear, actualtestY)

length of index_normalList 888
length of normalXfilenames 888
predict using one class svm: [-1 -1  1  1  1  1 -1  1  1 -1  1  1 -1  1 -1 -1  1  1 -1  1 -1 -1 -1 -1
  1 -1 -1  1  1 -1  1  1 -1 -1 -1  1 -1  1 -1 -1 -1 -1 -1  1 -1  1  1 -1
  1  1 -1  1 -1  1  1  1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1  1  1 -1 -1  1
 -1  1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1  1 -1  1  1  1 -1 -1  1 -1  1
 -1  1  1  1]
prediction of positive classes through base model [-1 -1  0 ...  3 -1  0]

Linear - F1 Scores
--------------------------------
Macro: 0.49470339917454537
Micro: 0.4882655981682885
Weighted: 0.48954644290505456
None: [0.49534884 0.46698113 0.56953642 0.56521739 0.43612335 0.43501326]

Evaluate

Overall accuracy: 0.4882655981682885
Accuracy for class -1: 0.49477351916376305
Accuracy for class 2: 0.5123152709359606
Accuracy for class 4: 0.47674418604651164
Accuracy for class 0: 0.47368421052631576
Accuracy for class 3: 0.4342105263157895
Accuracy for class 1: 0.581081081081081
macro f1 score: 0.4947

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatPoly)) if yhatPoly[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatPoly[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatPoly
  yhatPoly[index_yhat] = predClass

print("prediction of positive classes through base model", yhatPoly)

print("\nPoly - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatPoly, average='macro'))
print("Micro:", f1_score(actualtestY, yhatPoly, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatPoly, average='weighted'))
print("None:", f1_score(actualtestY, yhatPoly, average=None))

print("\nEvaluate\n")
evaluate2(yhatPoly, actualtestY)

length of index_normalList 870
length of normalXfilenames 870
predict using one class svm: [-1 -1  1  1  1  1 -1  1  1 -1  1  1 -1  1 -1 -1  1  1 -1  1 -1 -1 -1 -1
  1 -1 -1  1  1 -1  1  1 -1 -1 -1  1 -1  1 -1 -1 -1 -1 -1  1 -1 -1  1 -1
  1  1 -1  1 -1  1  1  1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1  1  1 -1 -1  1
 -1  1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1  1 -1  1  1  1 -1 -1  1 -1  1
 -1  1  1  1]
prediction of positive classes through base model [-1 -1  0 ...  3 -1  0]

Poly - F1 Scores
--------------------------------
Macro: 0.49621033019228894
Micro: 0.49170005724098453
Weighted: 0.49270120197491896
None: [0.50287687 0.4725537  0.57718121 0.5613079  0.42600897 0.43733333]

Evaluate

Overall accuracy: 0.49170005724098453
Accuracy for class -1: 0.5075493612078978
Accuracy for class 2: 0.5073891625615764
Accuracy for class 4: 0.47674418604651164
Accuracy for class 0: 0.47368421052631576
Accuracy for class 3: 0.4166666666666667
Accuracy for class 1: 0.581081081081081
macro f1 score: 0.49621

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatRBF)) if yhatRBF[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatRBF[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatRBF
  yhatRBF[index_yhat] = predClass

print("prediction of positive classes through base model", yhatRBF)

print("\nRBF - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatRBF, average='macro'))
print("Micro:", f1_score(actualtestY, yhatRBF, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatRBF, average='weighted'))
print("None:", f1_score(actualtestY, yhatRBF, average=None))

print("\nEvaluate\n")
evaluate2(yhatRBF, actualtestY)

length of index_normalList 875
length of normalXfilenames 875
predict using one class svm: [ 1 -1 -1 -1 -1  1  1 -1 -1  1  1 -1  1 -1  1 -1  1 -1  1  1  1 -1 -1 -1
 -1 -1  1  1 -1 -1 -1  1 -1  1 -1 -1  1 -1  1 -1 -1  1  1 -1 -1  1  1  1
  1  1  1  1 -1  1  1 -1 -1  1 -1  1  1  1 -1  1 -1 -1 -1 -1  1  1 -1  1
  1 -1 -1  1 -1  1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1
  1 -1 -1  1]
prediction of positive classes through base model [ 4 -1 -1 ... -1 -1 -1]

RBF - F1 Scores
--------------------------------
Macro: 0.47725813220152413
Micro: 0.5014310246136233
Weighted: 0.5004403650519041
None: [0.51125216 0.58371041 0.34782609 0.53513514 0.45333333 0.43229167]

Evaluate

Overall accuracy: 0.5014310246136233
Accuracy for class -1: 0.5145180023228804
Accuracy for class 2: 0.4876847290640394
Accuracy for class 4: 0.48255813953488375
Accuracy for class 0: 0.6172248803827751
Accuracy for class 3: 0.4473684210526316
Accuracy for class 1: 0.2702702702702703
macro f1 score: 0.477258132

In [ ]:
# get index of all 1s/normal class
index_normalList = [i for i in range(len(yhatSigmoid)) if yhatSigmoid[i]==1]
print("length of index_normalList", len(index_normalList))

testXfilenames = Xtestvalunknown + X_test
normalXfilenames = [testXfilenames[i] for i in index_normalList]
print("length of normalXfilenames", len(normalXfilenames))

normalLoader = SpecLoaderWnn(normalXfilenames, batch_size,spectrograms_path)
normal_predictions = model.predict_classes(x=normalLoader, batch_size=None)

print("predict using one class svm:",yhatSigmoid[:100])
for i in range(len(normal_predictions)):
  index_yhat = index_normalList[i]
  predClass = normal_predictions[i]
  # change the value in yhatSigmoid
  yhatSigmoid[index_yhat] = predClass

print("prediction of positive classes through base model", yhatSigmoid)

print("\nSigmoid - F1 Scores")
print("--------------------------------")
print("Macro:", f1_score(actualtestY, yhatSigmoid, average='macro'))
print("Micro:", f1_score(actualtestY, yhatSigmoid, average='micro'))
print("Weighted:", f1_score(actualtestY, yhatSigmoid, average='weighted'))
print("None:", f1_score(actualtestY, yhatSigmoid, average=None))

print("\nEvaluate\n")
evaluate2(yhatSigmoid, actualtestY)

length of index_normalList 895
length of normalXfilenames 895
predict using one class svm: [-1 -1  1  1  1  1 -1  1  1 -1  1  1  1  1 -1  1  1  1  1  1 -1 -1 -1 -1
  1 -1 -1  1  1 -1  1  1 -1 -1 -1  1 -1  1 -1 -1 -1 -1 -1  1 -1  1  1 -1
  1  1 -1  1 -1  1  1  1  1  1 -1  1 -1  1 -1 -1  1 -1 -1  1  1 -1 -1  1
 -1  1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1  1 -1 -1  1  1  1 -1 -1  1 -1  1
 -1  1  1  1]
prediction of positive classes through base model [-1 -1  0 ...  3 -1 -1]

Sigmoid - F1 Scores
--------------------------------
Macro: 0.4881275174744557
Micro: 0.48196908986834575
Weighted: 0.4827822349620117
None: [0.48686515 0.45520581 0.55263158 0.53369272 0.44631579 0.45405405]

Evaluate

Overall accuracy: 0.48196908986834575
Accuracy for class -1: 0.4843205574912892
Accuracy for class 2: 0.4876847290640394
Accuracy for class 4: 0.4883720930232558
Accuracy for class 0: 0.44976076555023925
Accuracy for class 3: 0.4649122807017544
Accuracy for class 1: 0.5675675675675675
macro f1 score: 0.4881